In [54]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.optimize import minimize
from python_module.pricing_model import BlackScholesModel, SABRModel

pd.options.display.float_format = '{:.2f}'.format

In [55]:
def compute_position_sizing(constraints, slide_df, expected_pnl, budget):

    def objective_function(weights, expected_pnl, budget):
        
        return (budget - expected_pnl.dot(weights))**2

    bnds = [(-1000, 1000)] * len(expected_pnl.index)

    x0 = [0] * len(expected_pnl.index)

    result = minimize(objective_function, x0, method='SLSQP', bounds=bnds,constraints=constraints, args=(expected_pnl, budget))

    summary = pd.concat([slide_df, expected_pnl], axis=1)
    summary['weights'] = result.x

    portfolio_slide = slide_df.multiply(result.x, axis=0).sum()
    portfolio_expected_pnl = expected_pnl.dot(result.x)

    print('portfolio slide')
    display(portfolio_slide)

    print(f'portfolio expected pnl {portfolio_expected_pnl:.2f}')

    return summary, result.x

In [56]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Explicit pricing parameters
S0 = F = 100
r = 0
option_type = 'put'
time_to_maturity = 250

# Market parameters
market_alpha = +0.15
market_beta  = +1.0
market_rho   = -0.9
market_nu    = 2.4

# Realized parameters
realized_alpha = +0.14
realized_beta  = +1.0
realized_rho   = -0.5
realized_nu    = 1.0
# Pre-processing
T = time_to_maturity / 250

# Pricing factory
market_data_list = list()
realized_data_list = list()

strike_delta_dict = {K: SABRModel.compute_option(F, K, T, market_alpha, market_beta, market_rho, market_nu, r, 'call')['delta'] for K in range(70, 130)}
strike_delta_s = pd.Series(strike_delta_dict)
delta_strike_dict = {delta: (strike_delta_s-delta).abs().idxmin() for delta in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]} 
strike_delta_s = pd.Series(delta_strike_dict).sort_values()
for K in strike_delta_s.values:

    option_type = 'call' if K > 100 else 'put'
    
    market_pricing_results = SABRModel.compute_option(F, K, T, market_alpha, market_beta, market_rho, market_nu, r, option_type, slide_list=[-0.03, 0.03])
    realized_pricing_results = SABRModel.compute_option(F, K, T, realized_alpha, realized_beta, realized_rho, realized_nu, r, option_type, slide_list=[-0.03, 0.03])
    
    market_data_list.append({
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **market_pricing_results})

    realized_data_list.append({
        'symbol': f"{time_to_maturity}_{K}_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **realized_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

realized_data_df = pd.DataFrame(realized_data_list)
realized_data_df = realized_data_df.set_index('symbol')

In [57]:
market_data_df

,option_type,time_to_maturity,S0,r,F,K,T,IV,price,delta,gamma,vega,theta,vanna,volga,slide pnl -0.03,slide pnl 0.03
symbol,,,,,,,,,,,,,,,,,
250_83_put,put,250,100,0,100,83,1.00,0.26,3.24,-0.20,0.01,0.28,-0.01,-0.64,54.00,0.81,-0.87
250_92_put,put,250,100,0,100,92,1.00,0.19,3.91,-0.30,0.02,0.35,-0.01,-0.64,34.05,0.86,-0.89
250_97_put,put,250,100,0,100,97,1.00,0.15,4.44,-0.39,0.03,0.38,-0.01,-0.34,9.52,0.87,-0.85
250_101_call,call,250,100,0,100,101,1.00,0.11,4.07,0.49,0.04,0.40,-0.01,0.51,1.57,0.88,-0.79
250_103_call,call,250,100,0,100,103,1.00,0.10,2.55,0.40,0.04,0.39,-0.01,1.44,37.61,0.88,-0.77
250_105_call,call,250,100,0,100,105,1.00,0.08,1.34,0.28,0.04,0.34,-0.01,2.78,159.67,0.83,-0.80
250_107_call,call,250,100,0,100,107,1.00,0.07,0.65,0.18,0.04,0.26,-0.00,3.67,339.72,0.67,-0.76


In [58]:
# -------
# SLIDE & EXPECTED PNL
# -------
slide_df = market_data_df[list(filter(lambda x: x.startswith('slide'), market_data_df.columns))]
expected_pnl = realized_data_df['price']-market_data_df['price']
expected_pnl.name = 'expected_pnl'

In [59]:
cons = [
    {'type': 'ineq', 'fun': lambda weights: slide_df.iloc[:, 1].dot(weights)},
    {'type': 'ineq', 'fun': lambda weights: slide_df.iloc[:, 0].dot(weights)}
]
budget = 50_000
summary, weights = compute_position_sizing(cons, slide_df, expected_pnl, budget)
summary

portfolio slide


slide pnl -0.03   157.35
slide pnl 0.03     -0.00
dtype: float64

portfolio expected pnl 7954.12


,slide pnl -0.03,slide pnl 0.03,expected_pnl,weights
symbol,,,,
250_83_put,0.81,-0.87,-1.42,-1000.00
250_92_put,0.86,-0.89,-0.65,-1000.00
250_97_put,0.87,-0.85,0.20,-1000.00
250_101_call,0.88,-0.79,1.15,372.72
250_103_call,0.88,-0.77,1.66,1000.00
250_105_call,0.83,-0.80,2.01,1000.00
250_107_call,0.67,-0.76,1.99,1000.00
